In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error

In [2]:
df_b = pd.read_csv("data_diabetes.csv")
df_b.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

In [3]:
for n in range(1, df_b.shape[1] - 1):
    print(df_b.columns[n], ": ", (df_b.iloc[:, n] == 0).sum())

Glucose :  5
BloodPressure :  35
SkinThickness :  227
Insulin :  374
BMI :  11
DiabetesPedigreeFunction :  0
Age :  0


In [4]:
df_b2 = df_b.copy()
for n in range(1, df_b.shape[1] - 1):
    df_b2 = df_b2.loc[df_b2.iloc[:, n] != 0, ]

In [5]:
len(df_b), len(df_b2)

(768, 392)

In [6]:
df_b2.agg(["min", "max", "mean"]).round(1)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
min           0.0     56.0           24.0            7.0     14.0  18.2   
max          17.0    198.0          110.0           63.0    846.0  67.1   
mean          3.3    122.6           70.7           29.1    156.1  33.1   

      DiabetesPedigreeFunction   Age  Outcome  
min                        0.1  21.0      0.0  
max                        2.4  81.0      1.0  
mean                       0.5  30.9      0.3

In [7]:
# df_b2["Outcome"].value_counts()
df_b2["Outcome"].value_counts(normalize = True).round(4) * 100

Outcome
0    66.84
1    33.16
Name: proportion, dtype: float64

In [9]:
df_train, df_test = train_test_split(df_b2, train_size = 0.8, random_state = 123)
len(df_train), len(df_test)

(313, 79)

In [10]:
df_train.head(1)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
16            0      118             84             47      230  45.8   

    DiabetesPedigreeFunction  Age  Outcome  
16                     0.551   31        1

In [11]:
model_nor_mm = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor_mm.transform(df_train)
arr_test_nor = model_nor_mm.transform(df_test)

In [12]:
arr_train_nor[:1, ]

array([[0.        , 0.43661972, 0.675     , 0.71428571, 0.25961538,
        0.56441718, 0.19957173, 0.16666667, 1.        ]])

In [13]:
pd.DataFrame([model_nor_mm.data_max_, model_nor_mm.data_min_],
             columns = df_train.columns)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0         17.0    198.0          110.0           63.0    846.0  67.1   
1          0.0     56.0           30.0            7.0     14.0  18.2   

   DiabetesPedigreeFunction   Age  Outcome  
0                     2.420  81.0      1.0  
1                     0.085  21.0      0.0

In [17]:
arr_train_nor.shape # 종속변수(Outcome)이 맨 뒤에 있음

(313, 9)

In [ ]:
model_knn1 = KNeighborsClassifier(n_neighbors = 3)
model_knn1.fit(X = arr_train_nor[:, :8], # train - 독립변수
               y = arr_train_nor[:, 8]) # train - 종속변수
pred_knn1 = model_knn1.predict(arr_test_nor[:, :8]) # test - 독립변수
pred_knn1[:3]

In [ ]:
arr_test_nor[:, 8]

In [21]:
from sklearn.metrics import accuracy_score, f1_score

In [22]:
accuracy_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn1)

0.759493670886076

In [23]:
f1_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn1)

0.5581395348837209

In [24]:
ls_k = [1, 3, 5, 7, 9, 11]
k = ls_k[0]
model_knn2 = KNeighborsClassifier(n_neighbors = k)
model_knn2.fit(X = arr_train_nor[:, :8],
               y = arr_train_nor[:, 8])
pred_knn2 = model_knn2.predict(arr_test_nor[:, :8])
val_acc = accuracy_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn2)
val_f1  = f1_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn2)
val_acc, val_f1

(0.6835443037974683, 0.46808510638297873)

In [ ]:
ls_k = [1, 3, 5, 7, 9, 11]

df_perf = pd.DataFrame()
for k in ls_k:
    print(k)
    model_knn2 = KNeighborsClassifier(n_neighbors = k)
    model_knn2.fit(X = arr_train_nor[:, :8],
                   y = arr_train_nor[:, 8])
    pred_knn2 = model_knn2.predict(arr_test_nor[:, :8])
    val_acc = accuracy_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn2)
    val_f1  = f1_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn2)
    
    df_perf_sub = pd.DataFrame(dict(k = [k], acc = [val_acc], f1 = [val_f1]))
    df_perf = pd.concat([df_perf, df_perf_sub])

In [39]:
# df_perf

In [43]:
best_k = df_perf.loc[df_perf["f1"] == df_perf["f1"].max(), "k"].values[0]
model_knn_b = KNeighborsClassifier(n_neighbors = best_k)
model_knn_b.fit(X = arr_train_nor[:, :8],
                y = arr_train_nor[:, 8])
pred_knn_b = model_knn_b.predict(arr_test_nor[:, :8])
val_acc = accuracy_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn_b)
val_f1  = f1_score(y_true = arr_test_nor[:, 8], y_pred = pred_knn_b)
val_acc, val_f1

(0.759493670886076, 0.5581395348837209)

In [44]:
df_dia = pd.read_csv("data_diamonds.csv")
df_dia.head(1)

carat    cut color clarity  depth  table  price     x     y     z
0   0.23  Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43

In [ ]:
df_dia_num = df_dia.select_dtypes(include = "number")
df_train, df_test = train_test_split(df_dia_num, train_size = 0.7, random_state = 12)
len(df_train), len(df_test)

In [47]:
from sklearn.neighbors import KNeighborsRegressor

In [53]:
model_knn_r1 = KNeighborsRegressor(n_neighbors = 3)
model_knn_r1.fit(df_train.drop(columns = "price"),
                 df_train["price"])
pred_knn_r1 = model_knn_r1.predict(df_test.drop(columns = "price"))
# pred_knn_r1[:4]

In [54]:
mean_absolute_error(y_true = df_test["price"], y_pred = pred_knn_r1) # MAE

866.3626663370825

In [55]:
from sklearn.tree import DecisionTreeRegressor

In [69]:
model_dt_r1 = DecisionTreeRegressor(max_depth = 6)
model_dt_r1.fit(X = df_train.drop(columns = "price"),
                y = df_train["price"])
pred_dt_r1 = model_dt_r1.predict(df_test.drop(columns = "price"))

In [70]:
mean_absolute_error(y_true = df_test["price"], y_pred = pred_dt_r1) # MAE

796.6261082640503

In [71]:
from sklearn.tree import DecisionTreeClassifier